<a href="https://colab.research.google.com/github/pujitha9908/Pujitha-/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [2]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [3]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [4]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [6]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [7]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [12]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [14]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [15]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?

yes, averaging the validation accuracy across multiple splits can give more consistent results. This technique is known as k-fold cross-validation.

In k-fold cross-validation, the dataset is divided into k subsets or folds. The model is trained on k-1 folds and evaluated on the remaining fold. This process is repeated k times, with each fold being used as the validation set once. The average validation accuracy across all k iterations is then calculated.

Averaging the validation accuracy across multiple splits can help to:

Reduce overfitting: By evaluating the model on multiple subsets of the data, you can get a more accurate estimate of its performance on unseen data.
Increase robustness: Averaging the validation accuracy across multiple splits can help to reduce the impact of outliers or noisy data points.
Improve generalizability: By evaluating the model on multiple subsets of the data, you can get a better sense of how well it will perform on new, unseen data.
However, it's important to note that k-fold cross-validation can be computationally expensive, especially for large datasets. Additionally, the choice of k can affect the results, and there is no one-size-fits-all solution for choosing the optimal value of k.

In general, k-fold cross-validation is a good technique to use when:

You have a small to medium-sized dataset.
You want to get a more accurate estimate of your model's performance.
You want to reduce overfitting and improve generalizability.
But, if you have a very large dataset, you may want to consider using other techniques, such as:

Holdout method: Where you split your data into a training set and a validation set, and evaluate your model on the validation set.
Bootstrap sampling: Where you create multiple subsets of your data by sampling with replacement, and evaluate your model on each subset.
Ultimately, the choice of technique will depend on the specific characteristics of your dataset and the goals of your project.


2. Does it give more accurate estimate of test accuracy?




Averaging the validation accuracy across multiple splits can give a more accurate estimate of the test accuracy, but it's not a guarantee.

Why it can be more accurate:

Reduced variance: By averaging the validation accuracy across multiple splits, you can reduce the variance of the estimate. This is because the validation accuracy can vary depending on the specific split of the data, and averaging across multiple splits can help to smooth out this variability.
More robust to overfitting: Averaging the validation accuracy across multiple splits can also help to reduce the impact of overfitting. If a model is overfitting to a particular split of the data, averaging across multiple splits can help to penalize this overfitting and give a more accurate estimate of the test accuracy.
Why it may not be more accurate:

Bias in the validation set: If the validation set is not representative of the test set, then averaging the validation accuracy across multiple splits may not give a more accurate estimate of the test accuracy. This can happen if the validation set is too small, or if it is not stratified to match the test set.
Model instability: If the model is not stable across different splits of the data, then averaging the validation accuracy may not give a more accurate estimate of the test accuracy. This can happen if the model is highly sensitive to the specific split of the data, or if it is not regularized properly.
When it is more likely to be accurate:

When the data is i.i.d.: If the data is independent and identically distributed (i.i.d.), then averaging the validation accuracy across multiple splits is more likely to give a more accurate estimate of the test accuracy.
When the model is stable: If the model is stable across different splits of the data, then averaging the validation accuracy is more likely to give a more accurate estimate of the test accuracy.
Code example

Here is an example of how you can use k-fold cross-validation to estimate the test accuracy in Python:

python

Verify

Open In Editor
Edit
Copy code
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# Define the model and the data
model = ...
X, y = ...

# Define the k-fold cross-validation object
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize the list to store the validation accuracy
val_acc = []

# Loop through each fold
for train_idx, val_idx in kfold.split(X):
    # Split the data into training and validation sets
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Train the model on the training set
    model.fit(X_train, y_train)

    # Evaluate the model on the validation set
    y_pred = model.predict(X_val)
    val_acc.append(accuracy_score(y_val, y_pred))

# Calculate the average validation accuracy
avg_val_acc = np.mean(val_acc)

print("Average validation accuracy:", avg_val_acc)
Note that this is just an example, and you may need to modify it to fit your specific use case.



3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?


The effect of the number of iterations (i.e., the number of folds in k-fold cross-validation) on the estimate of the test accuracy is a crucial aspect to consider.

Increasing the number of iterations:

Reduced variance: As you increase the number of iterations, the variance of the estimate decreases. This is because you are averaging the validation accuracy over more folds, which helps to smooth out the noise and variability in the data.
More robust estimate: With more iterations, you get a more robust estimate of the test accuracy. This is because the model is evaluated on a larger number of subsets of the data, which helps to reduce the impact of outliers and noisy data points.
Better representation of the data: As you increase the number of iterations, you are more likely to capture the underlying structure of the data. This is because each fold provides a slightly different view of the data, and averaging over more folds helps to provide a more comprehensive picture.
However, there are limits to this:

Computational cost: Increasing the number of iterations comes at a computational cost. As you increase the number of folds, the training and evaluation process becomes more time-consuming and resource-intensive.
Over-smoothing: If you increase the number of iterations too much, you may start to over-smooth the estimate. This means that the estimate becomes too averaged, and you lose the ability to capture the nuances and variability in the data.
Diminishing returns: At some point, increasing the number of iterations will yield diminishing returns. This means that the improvement in the estimate will become smaller and smaller as you add more iterations.
Rule of thumb:

A common rule of thumb is to use 5-10 folds for k-fold cross-validation. This provides a good balance between reducing variance and computational cost. However, the optimal number of iterations depends on the specific problem, dataset, and model.

When to use more iterations:

Small datasets: If you have a small dataset, you may want to use more iterations to reduce the variance of the estimate.
Noisy data: If your data is noisy or has outliers, you may want to use more iterations to help smooth out the estimate.
High-variance models: If you are using a model that has high variance (e.g., decision trees), you may want to use more iterations to reduce the impact of overfitting.
When to use fewer iterations:

Large datasets: If you have a large dataset, you may be able to get away with fewer iterations, as the law of large numbers will help to reduce the variance of the estimate.
Computational constraints: If you are working with limited computational resources, you may need to use fewer iterations to reduce the computational cost.
Simple models: If you are using a simple model (e.g., linear regression), you may not need as many iterations, as the model is less prone to overfitting.
In summary, increasing the number of iterations can provide a better estimate of the test accuracy, but there are limits to this. You need to balance the benefits of reduced variance and improved robustness with the computational cost and potential for over-smoothing.



4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


When dealing with a very small train dataset or validation dataset, increasing the number of iterations in k-fold cross-validation can help, but it's not a silver bullet. Here's why:

Small train dataset:

Overfitting: If the train dataset is very small, the model may overfit to the training data, even with k-fold cross-validation. Increasing the number of iterations can help, but it may not be enough to overcome the overfitting issue.
Limited representation: A small train dataset may not be representative of the underlying distribution of the data. Increasing the number of iterations can help to some extent, but it's still limited by the small size of the dataset.
Small validation dataset:

High variance: A small validation dataset can lead to high variance in the estimate of the test accuracy. Increasing the number of iterations can help to reduce this variance, but it may not be enough to provide a reliable estimate.
Unstable estimates: With a small validation dataset, the estimate of the test accuracy can be unstable and may fluctuate significantly between iterations. Increasing the number of iterations can help to stabilize the estimate, but it's still sensitive to the specific split of the data.
Increasing iterations can help, but...

While increasing the number of iterations can help to some extent, it's essential to recognize that it's not a substitute for a larger, more representative dataset. If the dataset is too small, increasing the number of iterations may not be enough to overcome the limitations of the data.

Other strategies to consider:

Data augmentation: If possible, consider using data augmentation techniques to increase the size of the dataset.
Transfer learning: If you have a small dataset, consider using pre-trained models and fine-tuning them on your dataset.
Ensemble methods: Consider using ensemble methods, such as bagging or boosting, to combine the predictions of multiple models trained on different subsets of the data.
Collect more data: If possible, collect more data to increase the size of the dataset.
In summary, while increasing the number of iterations in k-fold cross-validation can help with small datasets, it's not a panacea. It's essential to consider other strategies to overcome the limitations of small datasets.

































































































> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.